In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [ ]:
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index import SimpleDirectoryReader, GPTListIndex, LLMPredictor, PromptHelper, ServiceContext
from langchain import OpenAI

documents = SimpleDirectoryReader('./meeting_notes').load_data()

# set maximum input size
max_input_size = 4096
# set number of output tokens
num_outputs = 3000
# set maximum chunk overlap
max_chunk_overlap = 20
# set chunk size limit
chunk_size_limit = 600

prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit)
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

index = GPTListIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine(
    response_mode="tree_summarize"
)

summarization_query = "This text is a transcript from a virtual video meeting. Summarize the meeting, including the main points, decisions made, and action items." + \
                      "Use formatting to enhance the response. Use bulleted lists where applicable. Format the response in markdown." + \
                      "Format the response as follows: " + \
                      "Attendees:" + \
                      "Meeting Summary:" + \
                      "Main subjects:" + \
                      "Decisions made:" + \
                      "Action items:"
response = query_engine.query(summarization_query)

print(response)